# Rectangular grid vs Carstesian grid interpolation

In [2]:
import numpy as np
from scipy.ndimage import map_coordinates
from scipy.interpolate import RegularGridInterpolator


class CartesianGridInterpolator:
    def __init__(self, points, values, method='linear'):
        self.limits = np.array([[min(x), max(x)] for x in points])
        self.values = np.asarray(values, dtype=float)
        self.order = {'linear': 1, 'cubic': 3, 'quintic': 5}[method]

    def __call__(self, xi):
        """
        `xi` here is an array-like (an array or a list) of points.

        Each "point" is an ndim-dimensional array_like, representing
        the coordinates of a point in ndim-dimensional space.
        """
        # transpose the xi array into the ``map_coordinates`` convention
        # which takes coordinates of a point along columns of a 2D array.
        xi = np.asarray(xi).T

        # convert from data coordinates to pixel coordinates
        ns = self.values.shape
        coords = [(n-1)*(val - lo) / (hi - lo)
                  for val, n, (lo, hi) in zip(xi, ns, self.limits)]

        # interpolate
        return map_coordinates(self.values, coords,
                               order=self.order,
                               cval=np.nan)  # fill_value

In [26]:
x, y, z = np.linspace(-5, 5, 100), np.linspace(-5, 5, 100), np.linspace(-5, 5, 100)
xx, yy, zz = np.meshgrid(x, y, z, indexing='ij')
values = xx**3 + yy**3 + zz**3

rgi = RegularGridInterpolator((x, y, z), values, method='linear')
cgi = CartesianGridInterpolator((x, y, z), values, method='linear')

In [29]:
x_fine, y_fine, z_fine = np.linspace(-5, 5, 500), np.linspace(-5, 5, 500), np.linspace(-5, 5, 100)
coords = np.array(np.meshgrid(x_fine, y_fine, z_fine, indexing='ij')).reshape(3, -1).T

In [30]:
%%timeit
rgi_values = rgi(coords)

8.28 s ± 285 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [31]:
%%timeit
cgi_values = cgi(coords)

1.42 s ± 22.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
